In [2]:
# notebooks/recommendation_demo.ipynb

# Recommendation System Demo
# This notebook demonstrates how to use the recommendation system

import sys
import os
import pandas as pd

# Add the project root to Python path
sys.path.append('../')

# Import the recommendation system
from food-delivery-rec.recommendation_system import (
    RecommendationModelTrainer, 
    RecommendationEvaluator,
    RecommendationSystem
)

# Load your data
full_data = pd.read_csv('../data/test-train/full_data100k.csv')

print("🚀 Food Delivery Recommendation System Demo")
print("=" * 50)

# If you have data loaded, uncomment below:
trainer = RecommendationModelTrainer(full_data)
model_components = trainer.train_full_model()

SyntaxError: invalid syntax (3310949329.py, line 14)